# PRISMA Study Assessment Dataset Nanopublication Generator

This notebook generates a nanopublication declaring a PRISMA study assessment dataset for a systematic review.

**Template:** [Declaring a PRISMA study assessment dataset](https://w3id.org/np/RAwQj3SNiopwPrHXfoRT2JtYZSt-5JsDHjBDW6nYz_rDE) by Anne Fouilloux & Tobias Kuhn (2025-11-24)

## PRISMA 2020 Items Covered
- **Item 5:** Eligibility criteria
- **Item 11:** Risk of bias assessment methods
- **Item 17:** Study characteristics
- **Items 18-19:** Individual study results

## Workflow Position
1. PICO Research Question ✅
2. Search Strategy ✅
3. Search Execution Dataset ✅
4. Study Inclusion ✅ (one per study)
5. **Study Assessment Dataset** ← This notebook (one per review)

## Step 1: Configuration

Set the path to your JSON configuration file:

In [1]:
# Path to your JSON configuration file
ASSESSMENT_FILE = "../inputs/study-assessment-quantum-biodiversity.json"

## Step 2: Load Configuration

In [2]:
import json
from pathlib import Path

with open(ASSESSMENT_FILE, 'r') as f:
    config = json.load(f)

print("Configuration loaded successfully!")
print(f"\nAuthor: {config['author']['name']} ({config['author']['orcid']})")
print(f"Systematic Review: {config['systematic_review']}")
print(f"\nAssessment Dataset: {config['assessment']['label']}")
print(f"Creation Date: {config['assessment']['creation_date']}")
print(f"Dataset Location: {config['assessment']['dataset_file_location']}")

Configuration loaded successfully!

Author: Anne Fouilloux (0000-0002-1784-2920)
Systematic Review: https://w3id.org/np/RA8B3ptXUOsN7obpkFGtA0FBmsh0OnID53wOsUIpSKTcg

Assessment Dataset: Study Assessment Dataset: Quantum Computing Applications in Biodiversity Research
Creation Date: 2025-12-27
Dataset Location: https://zenodo.org/records/18070378


## Step 3: Setup Namespaces and Imports

In [3]:
from datetime import datetime, timezone
from rdflib import Graph, Dataset, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD, FOAF, PROV

# Define namespaces (DCT as Namespace to allow custom predicates)
NP = Namespace("http://www.nanopub.org/nschema#")
NPX = Namespace("http://purl.org/nanopub/x/")
NT = Namespace("https://w3id.org/np/o/ntemplate/")
ORCID = Namespace("https://orcid.org/")
SLV = Namespace("https://w3id.org/sciencelive/o/terms/")
DCT = Namespace("http://purl.org/dc/terms/")

# Temporary namespace for building
TEMP_NP = Namespace("http://purl.org/nanopub/temp/np/")

# Template URIs
STUDY_ASSESSMENT_TEMPLATE = URIRef("https://w3id.org/np/RAwQj3SNiopwPrHXfoRT2JtYZSt-5JsDHjBDW6nYz_rDE")
PROVENANCE_TEMPLATE = URIRef("https://w3id.org/np/RA7lSq6MuK_TIC6JMSHvLtee3lpLoZDOqLJCLXevnrPoU")
PUBINFO_TEMPLATE_1 = URIRef("https://w3id.org/np/RA0J4vUn_dekg-U1kK3AOEt02p9mT2WO03uGxLDec1jLw")
PUBINFO_TEMPLATE_2 = URIRef("https://w3id.org/np/RAukAcWHRDlkqxk7H2XNSegc1WnHI569INvNr-xdptDGI")

print("Namespaces configured.")

Namespaces configured.


## Step 4: Build the Nanopublication

In [4]:
# Create Dataset with namespace bindings
ds = Dataset()
ds.bind("this", "http://purl.org/nanopub/temp/np/")
ds.bind("sub", TEMP_NP)
ds.bind("np", NP)
ds.bind("dct", DCT)
ds.bind("nt", NT)
ds.bind("npx", NPX)
ds.bind("xsd", XSD)
ds.bind("rdfs", RDFS)
ds.bind("orcid", ORCID)
ds.bind("prov", PROV)
ds.bind("foaf", FOAF)
ds.bind("slv", SLV)

# URIs
np_uri = URIRef("http://purl.org/nanopub/temp/np/")
head_uri = TEMP_NP.Head
assertion_uri = TEMP_NP.assertion
provenance_uri = TEMP_NP.provenance
pubinfo_uri = TEMP_NP.pubinfo
dataset_uri = TEMP_NP.studyAssessmentDataset
author_uri = ORCID[config['author']['orcid']]
systematic_review_uri = URIRef(config['systematic_review'])

print("Dataset and URIs created.")

Dataset and URIs created.


In [5]:
# HEAD graph
head = ds.graph(head_uri)
head.add((np_uri, RDF.type, NP.Nanopublication))
head.add((np_uri, NP.hasAssertion, assertion_uri))
head.add((np_uri, NP.hasProvenance, provenance_uri))
head.add((np_uri, NP.hasPublicationInfo, pubinfo_uri))

print("HEAD graph created.")

HEAD graph created.


In [6]:
# ASSERTION graph
assertion = ds.graph(assertion_uri)
assessment = config['assessment']

# Type declaration
assertion.add((dataset_uri, RDF.type, SLV.StudyAssessmentDataset))

# Label
assertion.add((dataset_uri, RDFS.label, Literal(assessment['label'])))

# Link to systematic review
assertion.add((dataset_uri, DCT.isPartOf, systematic_review_uri))

# Creation date
assertion.add((dataset_uri, DCT.created, Literal(assessment['creation_date'], datatype=XSD.date)))

# Eligibility criteria (PRISMA Item 5)
assertion.add((dataset_uri, SLV.followsEligibilityCriteria, Literal(assessment['eligibility_criteria'])))

# Assessment technique (PRISMA Item 11)
assertion.add((dataset_uri, SLV.usesAssessmentTechnique, Literal(assessment['assessment_technique'])))

# Study characteristics (PRISMA Item 17)
assertion.add((dataset_uri, SLV.hasStudyCharacteristics, Literal(assessment['study_characteristics'])))

# Extraction method
assertion.add((dataset_uri, DCT.hasExtractionMethod, Literal(assessment['extraction_method'])))

# Study results (PRISMA Item 19)
assertion.add((dataset_uri, DCT.hasStudyResults, Literal(assessment['study_results'])))

# Quality assessment (PRISMA Item 18)
assertion.add((dataset_uri, DCT.hasQualityAssessment, Literal(assessment['quality_assessment'])))

# Dataset file location
assertion.add((dataset_uri, SLV.hasDatasetFileLocation, URIRef(assessment['dataset_file_location'])))

# Limitations (optional)
if assessment.get('limitations'):
    assertion.add((dataset_uri, SLV.hasLimitations, Literal(assessment['limitations'])))

print(f"ASSERTION graph created with {len(assertion)} triples.")

ASSERTION graph created with 12 triples.


In [7]:
# PROVENANCE graph
provenance = ds.graph(provenance_uri)
provenance.add((assertion_uri, PROV.wasAttributedTo, author_uri))

print("PROVENANCE graph created.")

PROVENANCE graph created.


In [8]:
# PUBINFO graph
pubinfo = ds.graph(pubinfo_uri)

# Author info
pubinfo.add((author_uri, FOAF.name, Literal(config['author']['name'])))

# Timestamp
timestamp = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
pubinfo.add((np_uri, DCT.created, Literal(timestamp, datatype=XSD.dateTime)))

# Creator
pubinfo.add((np_uri, DCT.creator, author_uri))

# License
pubinfo.add((np_uri, DCT.license, URIRef("https://creativecommons.org/licenses/by/4.0/")))

# Introduces the dataset
pubinfo.add((np_uri, NPX.introduces, dataset_uri))

# Created at
pubinfo.add((np_uri, NPX.wasCreatedAt, URIRef("https://nanodash.knowledgepixels.com/")))

# Label (truncated to 50 chars)
label = assessment['label'][:50] + "..." if len(assessment['label']) > 50 else assessment['label']
pubinfo.add((np_uri, RDFS.label, Literal(label)))

# Template references
pubinfo.add((np_uri, NT.wasCreatedFromTemplate, STUDY_ASSESSMENT_TEMPLATE))
pubinfo.add((np_uri, NT.wasCreatedFromProvenanceTemplate, PROVENANCE_TEMPLATE))
pubinfo.add((np_uri, NT.wasCreatedFromPubinfoTemplate, PUBINFO_TEMPLATE_1))
pubinfo.add((np_uri, NT.wasCreatedFromPubinfoTemplate, PUBINFO_TEMPLATE_2))

print("PUBINFO graph created.")

PUBINFO graph created.


## Step 5: Serialize and Save

In [9]:
# Serialize to TriG
trig_output = ds.serialize(format='trig')

# Save to file
output_path = Path(config['output']['filename'])
output_path.write_text(trig_output)

print(f"✅ Nanopub saved to: {output_path}")
print(f"\nAssertion contains {len(assertion)} triples")

✅ Nanopub saved to: quantum-biodiversity-study-assessment.trig

Assertion contains 12 triples


## Step 6: Display Output

In [10]:
print("Generated TriG output:")
print("=" * 70)
print(trig_output)

Generated TriG output:
@prefix dct: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix nt: <https://w3id.org/np/o/ntemplate/> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix slv: <https://w3id.org/sciencelive/o/terms/> .
@prefix sub: <http://purl.org/nanopub/temp/np/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:provenance {
    sub:assertion prov:wasAttributedTo orcid:0000-0002-1784-2920 .
}

sub:pubinfo {
    sub: rdfs:label "Study Assessment Dataset: Quantum Computing Applic..." ;
        dct:created "2025-12-27T18:01:06.231000+00:00"^^xsd:dateTime ;
        dct:creator orcid:0000-0002-1784-2920 ;
        dct:license <https://creativecommons.org/licenses/by/4.0/> ;
        npx:introduces sub:studyAssessmentDataset ;
        npx:wasCreatedAt <https:/

## Step 7: Validate

In [11]:
from nanopub import Nanopub, NanopubConf

conf = NanopubConf(use_test_server=True)

try:
    np_obj = Nanopub(rdf=output_path, conf=conf)
    print(f"✅ Validation passed!")
    print(f"   Assertion triples: {len(list(np_obj.assertion))}")
    print(f"   Provenance triples: {len(list(np_obj.provenance))}")
    print(f"   Pubinfo triples: {len(list(np_obj.pubinfo))}")
except Exception as e:
    print(f"❌ Validation error: {e}")

✅ Validation passed!
   Assertion triples: 12
   Provenance triples: 1
   Pubinfo triples: 11


## Step 8: Sign and Publish (Optional)

⚠️ **Warning:** Uncomment only when ready to publish.

In [12]:
# Sign and publish
PUBLISH = True
USE_TEST_SERVER = False

if PUBLISH:
    from nanopub import Nanopub, NanopubConf, load_profile
    
    profile = load_profile()
    print(f"Loaded profile: {profile.name}")
    
    conf = NanopubConf(profile=profile, use_test_server=USE_TEST_SERVER)
    np_obj = Nanopub(rdf=output_path, conf=conf)
    
    np_obj.sign()
    print(f"✓ Signed")
    
    np_obj.publish()
    print(f"✅ Published: {np_obj.source_uri}")
else:
    print("Publishing disabled. Set PUBLISH = True to enable.")

Loaded profile: Anne Fouilloux
✓ Signed
✅ Published: https://w3id.org/np/RAx_ZQScbvsz7Rvqk8scSYx06zojCc6Gjcvkxjj_MKwVM


---

## JSON Configuration Schema

```json
{
  "author": {
    "orcid": "0000-0002-1784-2920",
    "name": "Anne Fouilloux"
  },
  "systematic_review": "https://w3id.org/np/...",
  "assessment": {
    "label": "Study Assessment Dataset Label",
    "creation_date": "2025-01-15",
    "eligibility_criteria": "PRISMA Item 5: Detailed inclusion/exclusion criteria...",
    "assessment_technique": "PRISMA Item 11: Risk of bias tools and methods...",
    "study_characteristics": "PRISMA Item 17: Summary across included studies...",
    "extraction_method": "Data extraction methodology...",
    "study_results": "PRISMA Item 19: Individual study results...",
    "quality_assessment": "PRISMA Item 18: Risk of bias results...",
    "dataset_file_location": "https://zenodo.org/records/...",
    "limitations": "Optional: Assessment limitations..."
  },
  "output": {
    "filename": "my-study-assessment.trig"
  }
}
```

## PRISMA 2020 Mapping

| JSON Field | PRISMA Item | Description |
|------------|-------------|-------------|
| `eligibility_criteria` | Item 5 | Inclusion and exclusion criteria |
| `assessment_technique` | Item 11 | Risk of bias assessment methods |
| `study_characteristics` | Item 17 | Characteristics of included studies |
| `quality_assessment` | Item 18 | Risk of bias in individual studies |
| `study_results` | Item 19 | Results of individual studies |